In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

In [ ]:

file = 'filtered_data.csv'
df = pd.read_csv(file)

In [ ]:
print("First 5 rows of the dataset:")
display(df.head())

Summary Statistics

In [ ]:
display(df.describe())

Datatypes

In [ ]:
df.info()

In [ ]:
print(df.isnull().sum())
# No null values but if there were a few we would drop them
# df = df.dropna()

# Outliers #

In [ ]:
# Calculate and display outliers using IQR method for numerical columns
numerical_cols = df.select_dtypes(include=[np.number]).columns

for col in numerical_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    
    print(f"{col}:")
    print(f"  Outlier count: {len(outliers)}")
    print(f"  Outlier percentage: {len(outliers)/len(df)*100:.2f}%")
    print(f"  Range: [{lower_bound:.2f}, {upper_bound:.2f}]")
    if len(outliers) > 0:
        print(f"  Min outlier: {outliers[col].min():.2f}")
        print(f"  Max outlier: {outliers[col].max():.2f}")
    print()

In [ ]:
# Create visualizations for outliers
fig, axes = plt.subplots(3, 4, figsize=(20, 15))
axes = axes.flatten()

for i, col in enumerate(numerical_cols[:12]):  # Show first 12 numerical columns
    # Calculate outlier bounds for this column
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Create box plot
    sns.boxplot(data=df, y=col, ax=axes[i])
    axes[i].set_title(f'{col} - Outliers Detection')
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Create additional plots for remaining numerical columns if any
if len(numerical_cols) > 12:
    remaining_cols = numerical_cols[12:]
    n_remaining = len(remaining_cols)
    n_cols = 4
    n_rows = (n_remaining + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 5*n_rows))
    if n_rows == 1:
        axes = [axes] if n_remaining == 1 else axes
    else:
        axes = axes.flatten()
    
    for i, col in enumerate(remaining_cols):
        sns.boxplot(data=df, y=col, ax=axes[i])
        axes[i].set_title(f'{col} - Outliers Detection')
        axes[i].grid(True, alpha=0.3)
    
    # Hide empty subplots
    for j in range(i+1, len(axes)):
        axes[j].set_visible(False)
    
    plt.tight_layout()
    plt.show()

In [5]:
# --- Variable groups from dataset documentation ---
CONTINUOUS_VARS = [
    "age", "height", "weight", "sight_left", "sight_right",
    "SBP", "DBP", "BLDS", "tot_chole", "HDL_chole", "LDL_chole",
    "triglyceride", "hemoglobin", "serum_creatinine",
    "SGOT_AST", "SGOT_ALT", "gamma_GTP", "waistline"
]

ORDINAL_VARS = ["urine_protein"]  # exclude from IQR
CATEGORICAL_VARS = ["sex", "hear_left", "hear_right", "SMK_stat_type_cd", "DRK_YN"]

# Use intersection with actual columns
cont_cols = [c for c in CONTINUOUS_VARS if c in df.columns]

def iqr_bounds(s: pd.Series):
    s = pd.to_numeric(s, errors="coerce").dropna()
    q1, q3 = s.quantile([0.25, 0.75])
    iqr = q3 - q1
    low = q1 - 1.5 * iqr
    high = q3 + 1.5 * iqr
    return low, high

In [ ]:
# Remove (clear errors)

df_clean = df.copy()

REMOVE_RULES = {
    "waistline": lambda s: s > 200,  # e.g., 1000 cm entries
    "serum_creatinine": lambda s: s > 20,  # mg/dL, implausible highs
    "SGOT_AST": lambda s: s > 9000,  # extreme enzyme typos
    "SGOT_ALT": lambda s: s > 9000,
    "sight_left": lambda s: s >= 5,  # 10.0 eyesight entries -> remove
    "sight_right": lambda s: s >= 5,
}

# Apply removal rules
mask_keep = pd.Series(True, index=df_clean.index)
remove_report = []
for col, rule in REMOVE_RULES.items():
    if col in df_clean.columns:
        bad = rule(pd.to_numeric(df_clean[col], errors="coerce"))
        n_bad = int(bad.sum())
        if n_bad > 0:
            mask_keep &= ~bad
            remove_report.append((col, n_bad))

df_clean = df_clean.loc[mask_keep]
print("Rows removed (by column):", remove_report)

# =========================
# 2) CAP (winsorize at IQR)
# =========================
cap_report = []
for col in cont_cols:
    if col in df_clean.columns:
        low, high = iqr_bounds(df_clean[col])
        before = pd.to_numeric(df_clean[col], errors="coerce")
        n_low = int((before < low).sum())
        n_high = int((before > high).sum())
        df_clean[col] = before.clip(lower=low, upper=high)
        cap_report.append({
            "column": col,
            "capped_low": n_low,
            "capped_high": n_high
        })

print(pd.DataFrame(cap_report))

# ==============================
# 3) TRANSFORM (reduce skew with log1p)
# ==============================
LOG1P_VARS = [
    v for v in [
        "weight", "BLDS", "tot_chole", "HDL_chole", "LDL_chole",
        "triglyceride", "SGOT_AST", "SGOT_ALT", "gamma_GTP"
    ]
    if v in df_clean.columns
]

transform_report = []
for col in LOG1P_VARS:
    x = pd.to_numeric(df_clean[col], errors="coerce")
    n_neg = int((x < 0).sum())
    if n_neg:
        x = x.clip(lower=0)  # ensure non-negative before log1p
    df_clean[col] = np.log1p(x)
    transform_report.append({
        "column": col,
        "negatives_clipped": n_neg
    })

print(pd.DataFrame(transform_report))


# Feature extracting

Feature extraction after stabilizing data:

In [7]:
# Derived features (feature extraction)
df_clean['BMI'] = df_clean['weight'] / (df_clean['height'] / 100) ** 2
df_clean['pulse_pressure'] = df_clean['SBP'] - df_clean['DBP']
df_clean['LDL_to_HDL'] = df_clean['LDL_chole'] / df_clean['HDL_chole']
df_clean['TG_to_HDL'] = df_clean['triglyceride'] / df_clean['HDL_chole']
df_clean['sight_avg'] = (df_clean['sight_left'] + df_clean['sight_right']) / 2
df_clean['hearing_avg'] = (df_clean['hear_left'] + df_clean['hear_right']) / 2

Data Transformation

In [11]:
# Make a copy to avoid changing the original dataset
df_encoded = df_clean.copy()
df_encoded.drop(columns=['hear_left','hear_right','sight_left','sight_right'], inplace=True)

# Ordinal encoding: urine_protein (1 to 6 is ordered)
df_encoded["urine_protein"] = df_encoded["urine_protein"].astype(int)

# One-hot encoding for nominal variables
nominal_vars = ["sex", "SMK_stat_type_cd", "DRK_YN"]

df_encoded = pd.get_dummies(df_encoded, columns=nominal_vars, drop_first=True)

# Show only the newly created encoded columns
encoded_cols = [col for col in df_encoded.columns if any(var in col for var in nominal_vars)] + ["urine_protein"]
print(df_encoded[encoded_cols].head())


Feature scaling

In [26]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from IPython.display import display, Markdown

# Columns to show
cols_to_show = ["age", "weight", "waistline", "SBP", "triglyceride"]

# Before scaling
summary_before = df_encoded[cols_to_show].describe().round(2)

# After scaling
scaler = StandardScaler()
df_scaled = df_encoded.copy()
df_scaled[cols_to_show] = scaler.fit_transform(df_encoded[cols_to_show])
summary_after = df_scaled[cols_to_show].describe().round(2)

# Display summaries neatly in notebook
display(Markdown("### Summary statistics before scaling"))
display(summary_before)

display(Markdown("### Summary statistics after standardization (Z-score)"))
display(summary_after)


### Summary statistics before scaling

,age,weight,waistline,SBP,triglyceride
count,592819.00,592819.00,592819.0,592819.00,592819.00
mean,49.52,4.14,81.2,122.33,4.73
std,7.01,0.18,9.1,13.89,0.53
min,40.00,3.51,57.0,83.50,0.69
25%,45.00,4.03,75.0,112.00,4.36
50%,50.00,4.11,81.0,121.00,4.72
75%,55.00,4.26,87.0,131.00,5.11
max,60.00,4.54,105.0,159.50,5.70


### Summary statistics after standardization (Z-score)

,age,weight,waistline,SBP,triglyceride
count,592819.00,592819.00,592819.00,592819.00,592819.00
mean,0.00,0.00,-0.00,-0.00,-0.00
std,1.00,1.00,1.00,1.00,1.00
min,-1.36,-3.51,-2.66,-2.80,-7.64
25%,-0.64,-0.66,-0.68,-0.74,-0.71
50%,0.07,-0.19,-0.02,-0.10,-0.03
75%,0.78,0.65,0.64,0.62,0.71
max,1.50,2.18,2.62,2.68,1.82


Data splitting

In [ ]:
# Set 'DRK_YN' (Drinker or Not) as target variable — use the encoded dataframe
# If one-hot encoding created "DRK_YN_Y", use that as your target
target_col = "DRK_YN_Y" if "DRK_YN_Y" in df_encoded.columns else "DRK_YN"

# Separate features (X) and label (y)
X = df_encoded.drop(columns=[target_col])
y = df_encoded[target_col]

# Split the data into training and testing sets (80–20 split)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Training set shape (X_train):", X_train.shape)
print("Testing set shape (X_test):", X_test.shape)
print("Training set shape (y_train):", y_train.shape)
print("Testing set shape (y_test):", y_test.shape)

### Correlation filter

In [ ]:
# --- Step 1 Correlation-based filtering ---
corr_df = X_train.copy()
corr_df['target'] = y_train

corr_matrix = corr_df.corr(numeric_only=True)
corr_with_target = corr_matrix['target'].drop('target').sort_values(ascending=False)

plt.figure(figsize=(8,10))
sns.barplot(x=corr_with_target.values, y=corr_with_target.index, palette="viridis")
plt.title("Feature correlations with alcohol consumption (train set)")
plt.xlabel("Correlation coefficient")
plt.tight_layout()
plt.show()

selected_corr_features = corr_with_target[abs(corr_with_target) > 0.05].index.tolist()
print("Kept after correlation filter:", selected_corr_features)

In [ ]:
# --- SAMPLE for faster feature selection ---
# Use only 10–15% of data for RFE (same feature relationships, far less compute)
X_sample = X_train.sample(frac=0.15, random_state=42)
y_sample = y_train.loc[X_sample.index]

# --- LIGHTER RandomForest for selection ---
rf_estimator = RandomForestClassifier(
    n_estimators=300,  # fewer trees = faster
    random_state=42,
    n_jobs=-1
)

# --- RFE wrapper ---
rfe = RFE(estimator=rf_estimator, n_features_to_select=10, step=1)
rfe.fit(X_sample[selected_corr_features], y_sample)

selected_features = X_sample[selected_corr_features].columns[rfe.support_].tolist()
print("\nFinal selected features by RFE:\n", selected_features)

# --- Importance scores for selected features ---
importance_scores = pd.Series(
    rfe.estimator_.feature_importances_,
    index=X_sample[selected_corr_features].columns[rfe.support_]
).sort_values(ascending=False)

plt.figure(figsize=(8,6))
sns.barplot(x=importance_scores.values, y=importance_scores.index, palette="magma")
plt.title("Feature Importance (Random Forest - RFE, sample subset)")
plt.xlabel("Importance Score")
plt.tight_layout()
plt.show()

# Save selection for reproducibility
pd.Series(selected_features, name="selected_features").to_csv("selected_features.csv", index=False)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import pandas as pd

# Only train on selected features
fast_models = {
    "Logistic Regression": LogisticRegression(max_iter=500),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
}

results_fast = []

for name, model in fast_models.items():
    model.fit(X_train[selected_features], y_train)
    y_pred = model.predict(X_test[selected_features])
    y_prob = model.predict_proba(X_test[selected_features])[:,1] if hasattr(model, "predict_proba") else None

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob) if y_prob is not None else None

    results_fast.append({
        "Model": name,
        "Accuracy": acc,
        "F1": f1,
        "ROC-AUC": auc
    })

results_fast_df = pd.DataFrame(results_fast).sort_values(by="F1", ascending=False)
print(results_fast_df)

In [ ]:
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

slow_models = {
    "SVM (RBF Kernel)": SVC(kernel='rbf', probability=True, gamma='scale', random_state=42),
    "Neural Network": MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=200, early_stopping=True, random_state=42)
}

# Optional: downsample for faster runs
X_small = X_train[selected_features].sample(frac=0.1, random_state=42)
y_small = y_train.loc[X_small.index]

results_slow = []

for name, model in slow_models.items():
    model.fit(X_small, y_small)
    y_pred = model.predict(X_test[selected_features])
    y_prob = model.predict_proba(X_test[selected_features])[:,1] if hasattr(model, "predict_proba") else None

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob) if y_prob is not None else None

    results_slow.append({
        "Model": name,
        "Accuracy": acc,
        "F1": f1,
        "ROC-AUC": auc
    })

results_slow_df = pd.DataFrame(results_slow).sort_values(by="F1", ascending=False)
print(results_slow_df)